<a href="https://colab.research.google.com/github/Dipesh0987/Concepts-and-Technologies-of-AI/blob/main/DipeshKishorChhetri_worksheet10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
import pandas as pd
df = pd.read_csv("/content/drive/MyDrive/IMDB Dataset.csv")
length = len(df['review'])
print("Dataset info:")
print(f"Shape: {df.shape}")
print(f"\nColumn names: {df.columns.tolist()}")
print(f"\nFirst review example:\n{df['review'].iloc[0][:500]}...")
print(f"\nSentiment: {df['sentiment'].iloc[0]}")
print(f"\nSentiment distribution:")
print(df['sentiment'].value_counts())

Dataset info:
Shape: (50000, 2)

Column names: ['review', 'sentiment']

First review example:
One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ...

Sentiment: positive

Sentiment distribution:
sentiment
positive    25000
negative    25000
Name: count, dtype: int64


**Part 1:**
1. Load the dataset and preprocess the reviews.

    a. Convert all text to lowercase.

    b. Remove non-alphabetic characters (punctuation).

    c. Tokenize the reviews and remove common stopwords.

    d. Apply stemming to reduce words to their root form.

2. Split the dataset into training and testing sets (80% training, 20% testing).

3. Use a Naive Bayes classifier to classify the reviews into positive and negative categories.

    a. Implement a Bag-of-Words model using CountVectorizer.

    b. Train the Naive Bayes classifier using the training set.

In [14]:
# all reviews to lowercase
df['review'] = df['review'].str.lower()
print("After converting to lowercase:")
print(df['review'].iloc[0][:500])

After converting to lowercase:
one of the other reviewers has mentioned that after watching just 1 oz episode you'll be hooked. they are right, as this is exactly what happened with me.<br /><br />the first thing that struck me about oz was its brutality and unflinching scenes of violence, which set in right from the word go. trust me, this is not a show for the faint hearted or timid. this show pulls no punches with regards to drugs, sex or violence. its is hardcore, in the classic use of the word.<br /><br />it is called oz


In [15]:
import re

# Remove non-alphabetic characters and extra whitespace
df['review_clean'] = df['review'].apply(lambda x: re.sub(r'[^a-zA-Z\s]', '', x))
df['review_clean'] = df['review_clean'].apply(lambda x: re.sub(r'\s+', ' ', x).strip())

print("After removing non-alphabetic characters:")
print(df['review_clean'].iloc[0][:500])

After removing non-alphabetic characters:
one of the other reviewers has mentioned that after watching just oz episode youll be hooked they are right as this is exactly what happened with mebr br the first thing that struck me about oz was its brutality and unflinching scenes of violence which set in right from the word go trust me this is not a show for the faint hearted or timid this show pulls no punches with regards to drugs sex or violence its is hardcore in the classic use of the wordbr br it is called oz as that is the nickname g


In [16]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

# Get English stopwords
stop_words = set(stopwords.words('english'))

# Tokenize and remove stopwords
df['review_tokens'] = df['review_clean'].apply(lambda x: [word for word in x.split() if word not in stop_words])

print("After tokenization and removing stopwords:")
print(f"First review tokens (first 20): {df['review_tokens'].iloc[0][:20]}")
print(f"Number of tokens in first review: {len(df['review_tokens'].iloc[0])}")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


After tokenization and removing stopwords:
First review tokens (first 20): ['one', 'reviewers', 'mentioned', 'watching', 'oz', 'episode', 'youll', 'hooked', 'right', 'exactly', 'happened', 'mebr', 'br', 'first', 'thing', 'struck', 'oz', 'brutality', 'unflinching', 'scenes']
Number of tokens in first review: 170


In [17]:
from nltk.stem import PorterStemmer
nltk.download('punkt')

stemmer = PorterStemmer()

# Apply stemming to tokens
df['review_stemmed'] = df['review_tokens'].apply(lambda tokens: [stemmer.stem(word) for word in tokens])

print("After stemming:")
print(f"First review stemmed tokens (first 20): {df['review_stemmed'].iloc[0][:20]}")

# Join tokens back into strings for the model
df['review_processed'] = df['review_stemmed'].apply(lambda tokens: ' '.join(tokens))

print(f"\nProcessed review (first 500 chars):")
print(df['review_processed'].iloc[0][:500])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


After stemming:
First review stemmed tokens (first 20): ['one', 'review', 'mention', 'watch', 'oz', 'episod', 'youll', 'hook', 'right', 'exactli', 'happen', 'mebr', 'br', 'first', 'thing', 'struck', 'oz', 'brutal', 'unflinch', 'scene']

Processed review (first 500 chars):
one review mention watch oz episod youll hook right exactli happen mebr br first thing struck oz brutal unflinch scene violenc set right word go trust show faint heart timid show pull punch regard drug sex violenc hardcor classic use wordbr br call oz nicknam given oswald maximum secur state penitentari focus mainli emerald citi experiment section prison cell glass front face inward privaci high agenda em citi home manyaryan muslim gangsta latino christian italian irish moreso scuffl death stare


In [19]:
from sklearn.model_selection import train_test_split

# Prepare the features (X) and labels (y)
X = df['review_processed']  # Processed text
y = df['sentiment']  # Labels (positive/negative)

# Convert labels to binary (1 for positive, 0 for negative)
y_binary = y.map({'positive': 1, 'negative': 0})

# Split the data (80% training, 20% testing)
X_train, X_test, y_train, y_test = train_test_split(
    X, y_binary, test_size=0.2, random_state=42, stratify=y_binary
)

print("=== DATA SPLITTING RESULTS ===")
print(f"Total samples: {len(X)}")
print(f"Training samples: {len(X_train)} ({len(X_train)/len(X)*100:.1f}%)")
print(f"Testing samples: {len(X_test)} ({len(X_test)/len(X)*100:.1f}%)")
print(f"\nTraining set class distribution:")
print(y_train.value_counts())
print(f"\nTesting set class distribution:")
print(y_test.value_counts())

=== DATA SPLITTING RESULTS ===
Total samples: 50000
Training samples: 40000 (80.0%)
Testing samples: 10000 (20.0%)

Training set class distribution:
sentiment
1    20000
0    20000
Name: count, dtype: int64

Testing set class distribution:
sentiment
0    5000
1    5000
Name: count, dtype: int64


In [20]:
from sklearn.feature_extraction.text import CountVectorizer

# Initialize CountVectorizer
vectorizer = CountVectorizer(max_features=5000)  # Use top 5000 features

# Fit and transform on training data
X_train_bow = vectorizer.fit_transform(X_train)

# Transform test data
X_test_bow = vectorizer.transform(X_test)

print("=== BAG-OF-WORDS MODEL CREATED ===")
print(f"Vocabulary size: {len(vectorizer.vocabulary_)}")
print(f"Training data shape: {X_train_bow.shape}")
print(f"Testing data shape: {X_test_bow.shape}")
print(f"\nSample feature names (first 20):")
print(list(vectorizer.get_feature_names_out())[:20])

=== BAG-OF-WORDS MODEL CREATED ===
Vocabulary size: 5000
Training data shape: (40000, 5000)
Testing data shape: (10000, 5000)

Sample feature names (first 20):
['aaron', 'abandon', 'abc', 'abduct', 'abil', 'abl', 'abomin', 'abort', 'abound', 'aboutbr', 'abraham', 'abrupt', 'abruptli', 'absenc', 'absent', 'absolut', 'absorb', 'absurd', 'abund', 'abus']


In [21]:
from sklearn.naive_bayes import MultinomialNB

# Initialize the Naive Bayes classifier
nb_classifier = MultinomialNB()

# Train the classifier
nb_classifier.fit(X_train_bow, y_train)

print("=== NAIVE BAYES CLASSIFIER TRAINED ===")
print(f"Classifier type: {type(nb_classifier).__name__}")
print(f"Training completed successfully!")

=== NAIVE BAYES CLASSIFIER TRAINED ===
Classifier type: MultinomialNB
Training completed successfully!


**Part 2:**
1. Evaluate the performance of the model using the following metrics:

    a. Accuracy

    b. Precision, Recall, and F1-score

    c. Confusion Matrix

    d. ROC-AUC Score

In [22]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score, roc_curve
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [23]:
# Get predictions and prediction probabilities
y_pred = nb_classifier.predict(X_test_bow)
y_pred_proba = nb_classifier.predict_proba(X_test_bow)[:, 1]  # Probabilities for positive class

print("=== PREDICTIONS READY FOR EVALUATION ===")
print(f"Shape of predictions: {y_pred.shape}")
print(f"Shape of prediction probabilities: {y_pred_proba.shape}")

=== PREDICTIONS READY FOR EVALUATION ===
Shape of predictions: (10000,)
Shape of prediction probabilities: (10000,)


In [24]:
# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)

print("=== ACCURACY ===")
print(f"Accuracy: {accuracy:.4f}")
print(f"Accuracy percentage: {accuracy * 100:.2f}%")

=== ACCURACY ===
Accuracy: 0.8398
Accuracy percentage: 83.98%
